In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PyCon 2018:
Satellite data is for everyone: insights into modern remote sensing research
with open data and Python

"""
import os
from glob import glob
from keras.applications.vgg16 import VGG16 as VGG
from keras.applications.densenet import DenseNet201 as DenseNet
from keras.optimizers import SGD
from keras.layers import GlobalAveragePooling2D, Dense, Input, Conv2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from image_functions import simple_image_generator

# variables
path_to_split_datasets = "../data/ml_wrokflow/EuroSATallBandsTIF"
use_vgg = False
batch_size = 64

class_indices = {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2,
                 'Highway': 3, 'Industrial': 4, 'Pasture': 5,
                 'PermanentCrop': 6, 'Residential': 7, 'River': 8,
                 'SeaLake': 9}
num_classes = len(class_indices)

# contruct path
path_to_home = os.path.expanduser("~")
path_to_split_datasets = path_to_split_datasets.replace("~", path_to_home)
path_to_train = os.path.join(path_to_split_datasets, "train")
path_to_validation = os.path.join(path_to_split_datasets, "validation")


In [4]:

for path in (path_to_split_datasets, path_to_train, path_to_validation):
    cmd = f"tree {path}"
    get_ipython().system(cmd)

../data/ml_wrokflow/EuroSATallBandsTIF
├── train
│   ├── AnnualCrop
│   │   ├── AnnualCrop_1.tif
│   │   ├── AnnualCrop_10.tif
│   │   ├── AnnualCrop_1001.tif
│   │   ├── AnnualCrop_1003.tif
│   │   ├── AnnualCrop_1005.tif
│   │   ├── AnnualCrop_1006.tif
│   │   ├── AnnualCrop_1007.tif
│   │   ├── AnnualCrop_1008.tif
│   │   ├── AnnualCrop_1010.tif
│   │   ├── AnnualCrop_1011.tif
│   │   ├── AnnualCrop_1013.tif
│   │   ├── AnnualCrop_1014.tif
│   │   ├── AnnualCrop_1015.tif
│   │   ├── AnnualCrop_1016.tif
│   │   ├── AnnualCrop_1018.tif
│   │   ├── AnnualCrop_1020.tif
│   │   ├── AnnualCrop_1021.tif
│   │   ├── AnnualCrop_1022.tif
│   │   ├── AnnualCrop_1023.tif
│   │   ├── AnnualCrop_1024.tif
│   │   ├── AnnualCrop_1025.tif
│   │   ├── AnnualCrop_1026.tif
│   │   ├── AnnualCrop_1027.tif
│   │   ├── AnnualCrop_1029.tif
│   │   ├── AnnualCrop_1030.tif
│   │   ├── AnnualCrop_1031.tif
│   │   ├── AnnualCrop_1033.tif
│   │   ├── AnnualCrop_1034.tif
│   │   ├── AnnualCrop_1035.tif
│   │   ├

│   ├── Forest
│   │   ├── Forest_1.tif
│   │   ├── Forest_10.tif
│   │   ├── Forest_100.tif
│   │   ├── Forest_1000.tif
│   │   ├── Forest_1002.tif
│   │   ├── Forest_1006.tif
│   │   ├── Forest_1007.tif
│   │   ├── Forest_1009.tif
│   │   ├── Forest_101.tif
│   │   ├── Forest_1010.tif
│   │   ├── Forest_1012.tif
│   │   ├── Forest_1013.tif
│   │   ├── Forest_1014.tif
│   │   ├── Forest_1015.tif
│   │   ├── Forest_1016.tif
│   │   ├── Forest_1017.tif
│   │   ├── Forest_1018.tif
│   │   ├── Forest_1021.tif
│   │   ├── Forest_1022.tif
│   │   ├── Forest_1025.tif
│   │   ├── Forest_1027.tif
│   │   ├── Forest_1028.tif
│   │   ├── Forest_103.tif
│   │   ├── Forest_1030.tif
│   │   ├── Forest_1031.tif
│   │   ├── Forest_1032.tif
│   │   ├── Forest_1033.tif
│   │   ├── Forest_1034.tif
│   │   ├── Forest_1035.tif
│   │   ├── Forest_1036.tif
│   │   ├── Forest_1037.tif
│   │   ├── Forest_1038.tif
│   │   ├── Forest_1039.tif
│   │   ├── Forest_1040.tif
│   │   ├── Forest_1042.tif
│   │   ├── F

│   ├── HerbaceousVegetation
│   │   ├── HerbaceousVegetation_1.tif
│   │   ├── HerbaceousVegetation_1.tif.aux.xml
│   │   ├── HerbaceousVegetation_100.tif
│   │   ├── HerbaceousVegetation_1000.tif
│   │   ├── HerbaceousVegetation_1001.tif
│   │   ├── HerbaceousVegetation_1002.tif
│   │   ├── HerbaceousVegetation_1003.tif
│   │   ├── HerbaceousVegetation_1005.tif
│   │   ├── HerbaceousVegetation_1006.tif
│   │   ├── HerbaceousVegetation_1007.tif
│   │   ├── HerbaceousVegetation_1008.tif
│   │   ├── HerbaceousVegetation_1009.tif
│   │   ├── HerbaceousVegetation_101.tif
│   │   ├── HerbaceousVegetation_1011.tif
│   │   ├── HerbaceousVegetation_1012.tif
│   │   ├── HerbaceousVegetation_1014.tif
│   │   ├── HerbaceousVegetation_1015.tif
│   │   ├── HerbaceousVegetation_1016.tif
│   │   ├── HerbaceousVegetation_1017.tif
│   │   ├── HerbaceousVegetation_1019.tif
│   │   ├── HerbaceousVegetation_1021.tif
│   │   ├── HerbaceousVegetation_1023.tif
│   │   ├── HerbaceousVegetation_1024.tif
│   │

│   │   ├── HerbaceousVegetation_1632.tif
│   │   ├── HerbaceousVegetation_1633.tif
│   │   ├── HerbaceousVegetation_1634.tif
│   │   ├── HerbaceousVegetation_1636.tif
│   │   ├── HerbaceousVegetation_1638.tif
│   │   ├── HerbaceousVegetation_1639.tif
│   │   ├── HerbaceousVegetation_1640.tif
│   │   ├── HerbaceousVegetation_1641.tif
│   │   ├── HerbaceousVegetation_1643.tif
│   │   ├── HerbaceousVegetation_1644.tif
│   │   ├── HerbaceousVegetation_1647.tif
│   │   ├── HerbaceousVegetation_165.tif
│   │   ├── HerbaceousVegetation_1650.tif
│   │   ├── HerbaceousVegetation_1651.tif
│   │   ├── HerbaceousVegetation_1654.tif
│   │   ├── HerbaceousVegetation_1655.tif
│   │   ├── HerbaceousVegetation_1656.tif
│   │   ├── HerbaceousVegetation_1658.tif
│   │   ├── HerbaceousVegetation_1660.tif
│   │   ├── HerbaceousVegetation_1661.tif
│   │   ├── HerbaceousVegetation_1663.tif
│   │   ├── HerbaceousVegetation_1664.tif
│   │   ├── HerbaceousVegetation_1666.tif
│   │   ├── HerbaceousVegetation_16

│   │   ├── Highway_2461.tif
│   │   ├── Highway_2462.tif
│   │   ├── Highway_2464.tif
│   │   ├── Highway_2465.tif
│   │   ├── Highway_2466.tif
│   │   ├── Highway_2467.tif
│   │   ├── Highway_2470.tif
│   │   ├── Highway_2471.tif
│   │   ├── Highway_2472.tif
│   │   ├── Highway_2473.tif
│   │   ├── Highway_2474.tif
│   │   ├── Highway_2475.tif
│   │   ├── Highway_2476.tif
│   │   ├── Highway_2477.tif
│   │   ├── Highway_2478.tif
│   │   ├── Highway_248.tif
│   │   ├── Highway_2480.tif
│   │   ├── Highway_2481.tif
│   │   ├── Highway_2482.tif
│   │   ├── Highway_2483.tif
│   │   ├── Highway_2484.tif
│   │   ├── Highway_2487.tif
│   │   ├── Highway_2489.tif
│   │   ├── Highway_249.tif
│   │   ├── Highway_2490.tif
│   │   ├── Highway_2491.tif
│   │   ├── Highway_2494.tif
│   │   ├── Highway_2495.tif
│   │   ├── Highway_2496.tif
│   │   ├── Highway_2497.tif
│   │   ├── Highway_2498.tif
│   │   ├── Highway_2499.tif
│   │   ├── Highway_25.tif
│   │   ├── Highway_250.tif
│   │   ├── Highway

...

│   ├── Forest_2241.tif
│   ├── Forest_2244.tif
│   ├── Forest_2257.tif
│   ├── Forest_2258.tif
│   ├── Forest_2259.tif
│   ├── Forest_2261.tif
│   ├── Forest_2266.tif
│   ├── Forest_2270.tif
│   ├── Forest_2274.tif
│   ├── Forest_2280.tif
│   ├── Forest_2283.tif
│   ├── Forest_2286.tif
│   ├── Forest_2287.tif
│   ├── Forest_2288.tif
│   ├── Forest_2289.tif
│   ├── Forest_2298.tif
│   ├── Forest_2299.tif
│   ├── Forest_23.tif
│   ├── Forest_2302.tif
│   ├── Forest_2304.tif
│   ├── Forest_2305.tif
│   ├── Forest_2307.tif
│   ├── Forest_2308.tif
│   ├── Forest_2316.tif
│   ├── Forest_2319.tif
│   ├── Forest_232.tif
│   ├── Forest_2329.tif
│   ├── Forest_2331.tif
│   ├── Forest_2337.tif
│   ├── Forest_2344.tif
│   ├── Forest_235.tif
│   ├── Forest_2350.tif
│   ├── Forest_2353.tif
│   ├── Forest_2357.tif
│   ├── Forest_2362.tif
│   ├── Forest_2368.tif
│   ├── Forest_2370.tif
│   ├── Forest_2372.tif
│   ├── Forest_2374.tif
│   ├── Forest_2375.tif
│   ├── Forest_2379.tif
│   ├── Forest_2386.

├── Highway
│   ├── Highway_1.tif
│   ├── Highway_10.tif
│   ├── Highway_1001.tif
│   ├── Highway_1002.tif
│   ├── Highway_1005.tif
│   ├── Highway_1006.tif
│   ├── Highway_101.tif
│   ├── Highway_1010.tif
│   ├── Highway_1011.tif
│   ├── Highway_1013.tif
│   ├── Highway_1017.tif
│   ├── Highway_1024.tif
│   ├── Highway_1026.tif
│   ├── Highway_1027.tif
│   ├── Highway_103.tif
│   ├── Highway_1036.tif
│   ├── Highway_1039.tif
│   ├── Highway_1040.tif
│   ├── Highway_1042.tif
│   ├── Highway_1043.tif
│   ├── Highway_1044.tif
│   ├── Highway_1045.tif
│   ├── Highway_1049.tif
│   ├── Highway_1051.tif
│   ├── Highway_1055.tif
│   ├── Highway_106.tif
│   ├── Highway_1060.tif
│   ├── Highway_1061.tif
│   ├── Highway_1062.tif
│   ├── Highway_1066.tif
│   ├── Highway_1067.tif
│   ├── Highway_1072.tif
│   ├── Highway_1074.tif
│   ├── Highway_1075.tif
│   ├── Highway_1076.tif
│   ├── Highway_108.tif
│   ├── Highway_1080.tif
│   ├── Highway_1082.tif
│   ├── Highway_1084.tif
│   ├── Highway_1089.t

│   ├── Pasture_1028.tif
│   ├── Pasture_1029.tif
│   ├── Pasture_103.tif
│   ├── Pasture_1032.tif
│   ├── Pasture_1038.tif
│   ├── Pasture_1040.tif
│   ├── Pasture_1045.tif
│   ├── Pasture_105.tif
│   ├── Pasture_1050.tif
│   ├── Pasture_1053.tif
│   ├── Pasture_1054.tif
│   ├── Pasture_1057.tif
│   ├── Pasture_1064.tif
│   ├── Pasture_1066.tif
│   ├── Pasture_1069.tif
│   ├── Pasture_1073.tif
│   ├── Pasture_1079.tif
│   ├── Pasture_1085.tif
│   ├── Pasture_1090.tif
│   ├── Pasture_1097.tif
│   ├── Pasture_1098.tif
│   ├── Pasture_1099.tif
│   ├── Pasture_1105.tif
│   ├── Pasture_1107.tif
│   ├── Pasture_1109.tif
│   ├── Pasture_1111.tif
│   ├── Pasture_1112.tif
│   ├── Pasture_1113.tif
│   ├── Pasture_1116.tif
│   ├── Pasture_1126.tif
│   ├── Pasture_1127.tif
│   ├── Pasture_1128.tif
│   ├── Pasture_1129.tif
│   ├── Pasture_1131.tif
│   ├── Pasture_1136.tif
│   ├── Pasture_114.tif
│   ├── Pasture_1143.tif
│   ├── Pasture_1147.tif
│   ├── Pasture_1154.tif
│   ├── Pasture_1156.tif
│  

├── Residential
│   ├── Residential_1.tif
│   ├── Residential_1005.tif
│   ├── Residential_1009.tif
│   ├── Residential_1010.tif
│   ├── Residential_1013.tif
│   ├── Residential_1014.tif
│   ├── Residential_1015.tif
│   ├── Residential_1016.tif
│   ├── Residential_1018.tif
│   ├── Residential_102.tif
│   ├── Residential_1023.tif
│   ├── Residential_1025.tif
│   ├── Residential_1030.tif
│   ├── Residential_1037.tif
│   ├── Residential_1038.tif
│   ├── Residential_1049.tif
│   ├── Residential_1053.tif
│   ├── Residential_1054.tif
│   ├── Residential_106.tif
│   ├── Residential_1067.tif
│   ├── Residential_1069.tif
│   ├── Residential_107.tif
│   ├── Residential_1076.tif
│   ├── Residential_1079.tif
│   ├── Residential_1080.tif
│   ├── Residential_1086.tif
│   ├── Residential_109.tif
│   ├── Residential_1090.tif
│   ├── Residential_1092.tif
│   ├── Residential_1093.tif
│   ├── Residential_1097.tif
│   ├── Residential_1098.tif
│   ├── Residential_1103.tif
│   ├── Residential_1104.tif
│   ├

└── SeaLake
    ├── SeaLake_10.tif
    ├── SeaLake_1005.tif
    ├── SeaLake_1008.tif
    ├── SeaLake_1009.tif
    ├── SeaLake_1011.tif
    ├── SeaLake_102.tif
    ├── SeaLake_1022.tif
    ├── SeaLake_1025.tif
    ├── SeaLake_1026.tif
    ├── SeaLake_1031.tif
    ├── SeaLake_1037.tif
    ├── SeaLake_1046.tif
    ├── SeaLake_1052.tif
    ├── SeaLake_1055.tif
    ├── SeaLake_1056.tif
    ├── SeaLake_1057.tif
    ├── SeaLake_1058.tif
    ├── SeaLake_1059.tif
    ├── SeaLake_1062.tif
    ├── SeaLake_1065.tif
    ├── SeaLake_1068.tif
    ├── SeaLake_1070.tif
    ├── SeaLake_1071.tif
    ├── SeaLake_1072.tif
    ├── SeaLake_1073.tif
    ├── SeaLake_1074.tif
    ├── SeaLake_1075.tif
    ├── SeaLake_1076.tif
    ├── SeaLake_1085.tif
    ├── SeaLake_1089.tif
    ├── SeaLake_1094.tif
    ├── SeaLake_1095.tif
    ├── SeaLake_1099.tif
    ├── SeaLake_1101.tif
    ├── SeaLake_1104.tif
    ├── SeaLake_1107.tif
    ├── SeaLake_1108.tif
    ├── SeaLake_1118.tif
    ├── SeaLake_1119.tif
    ├── SeaLake_

In [5]:
# parameters for CNN
input_tensor = Input(shape=(64, 64, 13))
# introduce a additional layer to get from 13 to 3 input channels
input_tensor = Conv2D(3, (1, 1))(input_tensor)
if use_vgg:
    base_model_imagenet = VGG(include_top=False,
                              weights='imagenet',
                              input_shape=(64, 64, 3))
    base_model = VGG(include_top=False,
                     weights=None,
                     input_tensor=input_tensor)
    for i, layer in enumerate(base_model_imagenet.layers):
        # we must skip input layer, which has no weights
        if i == 0:
            continue
        base_model.layers[i+1].set_weights(layer.get_weights())
else:
    base_model_imagenet = DenseNet(include_top=False,
                                   weights='imagenet',
                                   input_shape=(64, 64, 3))
    base_model = DenseNet(include_top=False,
                          weights=None,
                          input_tensor=input_tensor)
    for i, layer in enumerate(base_model_imagenet.layers):
        # we must skip input layer, which has no weights
        if i == 0:
            continue
        base_model.layers[i+1].set_weights(layer.get_weights())


Instructions for updating:
Colocations handled automatically by placer.
74842112/74836368 [==============================] - 19s 0us/step


In [6]:
# add a global spatial average pooling layer
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
# or just flatten the layers
# top_model = Flatten()(top_model)

# let's add a fully-connected layer
if use_vgg:
    # only in VGG19 a fully connected nn is added for classfication
    # DenseNet tends to overfitting if using additionally dense layers
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
# and a logistic layer
predictions = Dense(num_classes, activation='softmax')(top_model)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# print network structure
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 13)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 3)    42          input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 70, 70, 3)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (

In [10]:
!pip3 install scikit-image

     |████████████████████████████████| 30.1MB 5.9MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 5.0MB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 5.2MB/s eta 0:00:01
  Stored in directory: /Users/shoaib/Library/Caches/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx


In [11]:
# defining ImageDataGenerators
# ... initialization for training
training_files = glob(path_to_train + "/**/*.tif")
train_generator = simple_image_generator(training_files, class_indices,
                                         batch_size=batch_size,
                                         rotation_range=45,
                                         horizontal_flip=True,
                                         vertical_flip=True)

# ... initialization for validation
validation_files = glob(path_to_validation + "/**/*.tif")
validation_generator = simple_image_generator(validation_files, class_indices,
                                              batch_size=batch_size)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False
# set convolution block for reducing 13 to 3 layers trainable
for layer in model.layers[:2]:
        layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# generate callback to save best model w.r.t val_categorical_accuracy
if use_vgg:
    file_name = "vgg"
else:
    file_name = "dense"
checkpointer = ModelCheckpoint("../data/models/" + file_name +
                               "_ms_transfer_init." +
                               "{epoch:02d}-{val_categorical_accuracy:.3f}." +
                               "hdf5",
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True,
                               mode='max')
earlystopper = EarlyStopping(monitor='valrw_categorical_accuracy',
                             patience=10,
                             mode='max',
                             restore_best_weights=True)
history = model.fit_generator(train_generator,
                              steps_per_epoch=1000,
                              epochs=10000,
                              callbacks=[checkpointer, earlystopper],
                              validation_data=validation_generator,
                              validation_steps=500)
initial_epoch = len(history.history['loss'])+1

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
names = []
for i, layer in enumerate(model.layers):
    names.append([i, layer.name, layer.trainable])
print(names)

if use_vgg:
    # we will freaze the first convolutional block and train all
    # remaining blocks, including top layers.
    for layer in model.layers[:2]:
        layer.trainable = True
    for layer in model.layers[2:5]:
        layer.trainable = False
    for layer in model.layers[5:]:
        layer.trainable = True
else:
    for layer in model.layers[:2]:
        layer.trainable = True
    for layer in model.layers[2:8]:
        layer.trainable = False
    for layer in model.layers[8:]:
        layer.trainable = True



Epoch 1/10000
 553/1000 [===============>..............] - ETA: 1:21:37 - loss: 0.6640 - categorical_accuracy: 0.7822

KeyboardInterrupt: 

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# generate callback to save best model w.r.t val_categorical_accuracy
if use_vgg:
    file_name = "vgg"
else:
    file_name = "dense"
checkpointer = ModelCheckpoint("../data/models/" + file_name +
                               "_ms_transfer_final." +
                               "{epoch:02d}-{val_categorical_accuracy:.3f}" +
                               ".hdf5",
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True,
                               mode='max')
earlystopper = EarlyStopping(monitor='val_categorical_accuracy',
                             patience=10,
                             mode='max')
model.fit_generator(train_generator,
                    steps_per_epoch=1000,
                    epochs=10000,
                    callbacks=[checkpointer, earlystopper],
                    validation_data=validation_generator,
                    validation_steps=500,
                    initial_epoch=initial_epoch)
